In [0]:
spark.sql("""select crs_dep_time,
                    dep_time,
                    wheels_on,
                    crs_arr_time,
                    arr_time
           from dev.spark_db.flight_time_raw""").limit(10).display()

# 1115 -> 11:15
# 650 -> 06:50


In [0]:
flight_raw_df = spark.read.table("dev.spark_db.flight_time_raw")

In [0]:
from pyspark.sql.functions import expr
flight_raw_df_step1 =  flight_raw_df.withColumns(
        {
            "crs_dep_time_hh" : expr("left(lpad(crs_dep_time , 4, 0),2)"),
            "crs_dep_time_mm" : expr("right(lpad(crs_dep_time , 4, 0),2)"),
        }

)

In [0]:
flight_raw_df_step1.limit(5).display()

In [0]:
flight_raw_df_step2 =  flight_raw_df_step1.withColumns(
        {
            "crs_dep_time_interval" : expr("cast(concat(crs_dep_time_hh , ':' , crs_dep_time_mm) as INTERVAL HOUR TO MINUTE )")
        }

)
flight_raw_df_step2.limit(3).display()

In [0]:
from pyspark.sql.functions import expr
flight_raw_df_step2 =  flight_raw_df_step1.withColumns(
        {
            "crs_dep_time_interval" : expr("crs_dep_time_hh || ':' || crs_dep_time_mm")
        }

)

In [0]:
flight_raw_df_step2.limit(3).display()

Develop Resuable Function

In [0]:
def interval_hhmm(col_name):
        from pyspark.sql.functions import expr

        return (f"""expr(
            

                cast(concat( left(lpad({col_name} , 4, '0'), 2) , ':' ,right(lpad({col_name} , 4, '0'), 2))) as INTERVAL HOUR TO MINUTE
                
                
                )""" 
        )

In [0]:
def interval_hhmm(col_name):
        from pyspark.sql.functions import expr

        return expr(f"""(
            

                cast(concat( left(lpad({col_name} , 4, '0'), 2) , ':' ,right(lpad({col_name} , 4, '0'), 2)) as INTERVAL HOUR TO MINUTE)
                
                
                )""" 
        )

flight_raw_df_step12 =   (  
      
      flight_raw_df.withColumns
      (
        {
            "CRS_DEP_TIME" : interval_hhmm('CRS_DEP_TIME'),
            "DEP_TIME" : interval_hhmm('DEP_TIME'),
            "WHEELS_ON" : interval_hhmm('WHEELS_ON'),
            "CRS_ARR_TIME" : interval_hhmm('CRS_ARR_TIME'),
            "ARR_TIME" : interval_hhmm('ARR_TIME'),
            "TAXI_IN" : expr(" cast(taxi_in as INTERVAL MINUTE)")
        }

      )
)

flight_raw_df_step12.limit(3).display()

In [0]:
flight_raw_df_step12.write.mode('overwrite').saveAsTable("dev.spark_db.flight_time")